In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # gráficos dinâmicos
import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima.arima import auto_arima
from prophet import Prophet

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')
dataset = pd.read_csv('acoes.csv', parse_dates=['Date'], index_col='Date', date_format=dateparse, usecols=['Date', 'BOVA'])


In [ ]:
time_series = dataset['BOVA']
display(time_series)

In [ ]:
time_series.index.max()


In [ ]:
time_series.index.min()

In [ ]:
plt.plot(time_series);

In [ ]:
figura = px.line(title='Histórico do preço das ações')
figura.add_scatter(x=time_series.index, y=time_series)


In [ ]:
decomposicao = seasonal_decompose(time_series, period=723) # Os dado precisam ser lidos na ida e na volta, então o periodo é metade do tamanho total da base de dados
tendencia = decomposicao.trend
sazonal= decomposicao.seasonal
aleatorio = decomposicao.resid

In [ ]:
plt.plot(tendencia);


In [ ]:
plt.plot(sazonal)

In [ ]:
plt.plot(aleatorio)

In [ ]:
modelo = auto_arima(time_series, suppress_warnings=True, error_action='ignore')
# Parâmetros P, Q e D
modelo.order

In [ ]:
previsoes = modelo.predict(n_periods=90) # Dias para frente
previsoes # Previsão de time_series + 90


In [ ]:
len(time_series)

In [ ]:
treinamento = time_series[:1081]
treinamento.shape


In [ ]:
teste = time_series[1081:]
teste.shape

In [ ]:
teste.index.min(), teste.index.max()

In [ ]:
modelo2 = auto_arima(treinamento, suppress_warnings=True, error_action='ignore')
display(teste.index)
len(teste.index)

In [ ]:
# Fazendo as previsoes para os próximos 365 dias com base nos primeiros 1081 dias dos dados em treinamentos
previsoes = modelo2.predict(n_periods=365)
previsoes = pd.DataFrame(previsoes)
previsoes.index = teste.index
previsoes.columns = ['Previsões']
previsoes

In [ ]:
teste

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(treinamento, label = 'Treinamento')
plt.plot(teste, label = 'Teste')
plt.plot(previsoes, label = 'Previsões')
plt.legend();
# Nota-se que ARIMA não é bom para séries temporais e dados financeiros
# É mais eficiente quando são dados relacionados com o tempo

In [ ]:
sum(abs(teste - previsoes['Previsões']) / len(teste))
# Em média, se erra o preço das ações em R$ 10,26 para cima ou para baixo. O objetivo é diminuir ao máximo esse erro. 


In [ ]:
# O código abaixo faz a mesma coisa
from sklearn.metrics import mean_absolute_error
mean_absolute_error(teste, previsoes)

In [ ]:
dataset = pd.read_csv('acoes.csv', usecols=['Date', 'BOVA'])
dataset

In [ ]:
dataset= dataset[['Date', 'BOVA']].rename(columns = {'Date': 'ds', 'BOVA': 'y'})
dataset

In [ ]:
modelo = Prophet()
modelo.fit(dataset)

In [48]:
futuro = modelo.make_future_dataframe(periods=90)
previsoes = modelo.predict(futuro)

In [ ]:
previsoes.head()

In [ ]:
len(dataset), len(previsoes)

In [ ]:
modelo.plot(previsoes, xlabel = 'Data', ylabel = 'Preço')

In [ ]:
modelo.plot_components(previsoes)

In [49]:
# Criando gráficos dinâmicos
from prophet.plot import plot_plotly, plot_components_plotly

In [50]:
plot_plotly(modelo, previsoes)
# arima olha os dados e devolve uma relação linear do que ele observou, fazendo uma previsão linear.
# prophet tenta imitar a base de dados original o máximo possível (e ainda estende para a quantidade de dias que eu quiser). Não é linha reta como arima.

In [46]:
plot_components_plotly(modelo, previsoes)

In [45]:
# Essa variável previsoes abaixo não é a mesma de cima. Acima era uma previsão para 90 dias além da base de dados original.
# Abaixo é imitando o mais proximo possivel a base original dataset, sem fazer previsão para o futuro
pred = modelo.make_future_dataframe(periods=0)
previsoes = modelo.predict(pred) #está fazendo previsões para a mesma quantidade de registros em dataset

In [ ]:
previsoes.shape # tem o mesmo tamanho de dataset

In [ ]:
previsoes = previsoes['yhat'].tail(365) # mudando o tamanho de previsoes

In [ ]:
previsoes # Agora só tem 365 registros para comparar com teste

In [ ]:
mean_absolute_error(teste, previsoes)
# Erro foi menor, o que indica que Prophet é melhor